# Lecture 9.5: More on Tidy Data



With this I plan to convince you that **tidy data** is the way to go.

 

This notebook demonstrates:



1. How to convert *wide* to **tidy** format.

2. How **filtering, grouping, and scaling** is more complicated in **wide** format compared to **tidy**.

3. How **visualization** in Altair is also simpler with **tidy** data.



---

## 1. Imports and Setup

In [ ]:
import pandas as pd
import altair as alt

## 2. Creating the Example Dataset (Wide Format)



We start with a **wide** DataFrame: each region (`North`, `South`, `East`, `West`) is in its own column.

In [ ]:
wide_df = pd.DataFrame({
    "Year":  [2020, 2021, 2022, 2023],
    "North": [100, 150, 200, 250],
    "South": [ 90, 130, 170, 220],
    "East":  [ 80, 120, 160, 210],
    "West":  [ 70, 110, 150, 200]
})

display(wide_df)


## 3. Converting Wide to Tidy



We use `pd.melt()` to **unpivot** the data so that each row represents a `(Year, Region, Sales)` combination.

In [ ]:
tidy_df = wide_df.melt(
    id_vars=["Year"], 
    var_name="Region", 
    value_name="Sales"
)

display(tidy_df)


# 4. Filtering & Grouping



We'll do two demonstrations:

1. **Filtering** data for a specific region and year range.

2. **Scaling** each region's values by its mean sales.



---

## 4.1 Filtering for the East region where `Year > 2021`

### 4.1.1 Wide Format (Complicated)



Since there's **no direct "Region" column**, we must manually pick the column (`East`) and rename it:

In [ ]:
east_after_2021_wide = wide_df.loc[wide_df["Year"] > 2021, ["Year", "East"]]
# east_after_2021_wide = east_after_2021_wide.rename(columns={"East": "Sales"})
display(east_after_2021_wide)


### 4.1.2 Tidy Format (Easy)



Just use `query("Region == 'East' and Year > 2021")`:

In [ ]:
east_after_2021_tidy = tidy_df.query("Region == 'East' and Year > 2021")

display(east_after_2021_tidy)


## 4.2 Grouping & Scaling by the Mean



**Task**: Divide each region's sales by that region's **mean** sales.



### 4.2.1 Wide Format



Compute the mean of each column and then **manually** scale:

In [ ]:
import numpy as np

mean_north = wide_df["North"].mean()
mean_south = wide_df["South"].mean()
mean_east  = wide_df["East"].mean()
mean_west  = wide_df["West"].mean()

scaled_wide = wide_df.copy()
scaled_wide["North"] = scaled_wide["North"] / mean_north
scaled_wide["South"] = scaled_wide["South"] / mean_south
scaled_wide["East"]  = scaled_wide["East"]  / mean_east
scaled_wide["West"]  = scaled_wide["West"]  / mean_west

# a for could be used to avoid repetition, but it's still cumbersome

display(scaled_wide)


### 4.2.2 Tidy Format



A single `.groupby("Region")` and `.transform()` handles **all** regions:

In [ ]:
scaled_tidy = tidy_df.copy()
scaled_tidy["Sales_Scaled"] = scaled_tidy.groupby("Region")["Sales"] \
                                         .transform(lambda x: x / x.mean())

display(scaled_tidy)


# 5. Visualization in Altair



We'll create a simple line plot of **Sales** over **Year** for each region.



---



## 5.1 Wide Format



### Option A: Layer each region's line separately

In [ ]:
chart_north = alt.Chart(wide_df).mark_line(stroke='blue').encode(
    x="Year:O",
    y="North:Q"
).properties(title="North")

chart_south = alt.Chart(wide_df).mark_line(stroke='red').encode(
    x="Year:O",
    y="South:Q"
).properties(title="South")

chart_east = alt.Chart(wide_df).mark_line(stroke='green').encode(
    x="Year:O",
    y="East:Q"
).properties(title="East")

chart_west = alt.Chart(wide_df).mark_line(stroke='orange').encode(
    x="Year:O",
    y="West:Q"
).properties(title="West")

# compose the charts
chart_wide_layered = alt.layer(chart_north, chart_south, chart_east, chart_west).properties(
    width=400,
    height=300
)

# Adding the legends would be a bit more work

chart_wide_layered


- We had to **manually** define a chart for each column (region).

- Adding new regions or removing one requires extra lines of code.

- Legends and other customizations would be more complex.



### Option B: Use `transform_fold` to pivot columns inside Altair



This is effectively using Altair to do what `melt()` does, but you must list **all** columns:

In [ ]:
chart_wide_pivot = (
    alt.Chart(wide_df)
    .transform_fold( # this is like pd.melt()!!!!
        fold=["North","South","East","West"],  # must list every region
        as_=["Region","Sales"]
    )
    .mark_line()
    .encode(
        x="Year:O",
        y="Sales:Q",
        color="Region:N"
    )
    .properties(
        width=400,
        height=300
    )
)

chart_wide_pivot


## 5.2 Tidy Format (So Much Simpler)



A single command, no need to list the regions. Altair **automatically** creates multiple lines, coloring by `Region`.

In [ ]:
chart_tidy = alt.Chart(tidy_df).mark_bar().encode(
    x="Year:O",
    y="Sales:Q",
    color="Region:N"
).properties(
    width=400,
    height=300
).interactive()

chart_tidy


# 6. Final Comparison


| **Action**               | **Wide Format**                                                       | **Tidy Format**                                                  |
|:-------------------------|:----------------------------------------------------------------------|:-----------------------------------------------------------------|
| **Filtering**            | Must pick & rename columns, no direct 'Region' column                 | Simple `query("Region=='X'")`                                    |
| **Grouping or Scaling**  | Repeat operations for each column (or do complex loops)               | Single `.groupby("Region")` call                                 |
| **Adding New Categories**| Must add **new columns** and update code                              | Rows just **grow**, existing code continues to work              |
| **Visualization**        | Either layer each column manually or use `transform_fold`             | One-liner with `color="Region:N"`                                |
| **Code Complexity**      | High (lots of repeated steps, listing columns)                        | Low (concise, flexible)                                          |


**Takeaway**: 

- **Tidy format** is recommended for most data science tasks because it avoids repetitive code, makes grouping/filtering straightforward, and integrates smoothly with visualization libraries like Altair, seaborn, etc.

- **Wide format** can be okay for quick tasks or certain machine learning APIs, but it often becomes cumbersome when you need to filter, group, scale, or plot multiple categories.



**Whenever possible, convert to tidy** to save time and headaches!